In [1]:
import numpy as np 
import pandas as pd

import plotly
import geopandas 
# import pyshp 
import shapely

import plotly.figure_factory as ff

import requests
import json

import prettytable

import time

import warnings

from functools import reduce

# PPP Data (X)

First, we get the PPP loan by loan data from the SBA

PPP is by zip code 

We need census which is by census track 

Labor outcomes are by county 

In [8]:
#all of this is to just get the raw data 

base_url = 'https://data.sba.gov/dataset/8aa276e2-6cab-4f86-aca4-a7dde42adf24/resource/501af711-1c91-477a-80ce-bf6428eb9253/download/public_150k_plus_220403.csv'

urls = [base_url]

for i in range(1,13):
    num_url = 'https://data.sba.gov/dataset/8aa276e2-6cab-4f86-aca4-a7dde42adf24/resource/85f66605-568a-4400-aa5c-3803873153dc/download/public_up_to_150k_{}_220403.csv'.format(i)
    urls.append(num_url)
    
dfs = []
for url in urls: 
    dfs.append(pd.read_csv(url))
    
for n, df in enumerate(dfs): 
    df.to_csv('/Users/zachmariani/ECMA/ECMA_Final_Project/Data/PPP/ppp_{}.csv'.format(n))
    
full_ppp_df = pd.concat(dfs, axis = 1)
full_ppp_df.to_csv('/Users/zachmariani/ECMA/ECMA_Final_Project/Data/PPP/clean_ppp.csv')



In [9]:
pd.read_csv('https://data.sba.gov/dataset/8aa276e2-6cab-4f86-aca4-a7dde42adf24/resource/85f66605-568a-4400-aa5c-3803873153dc/download/public_up_to_150k_1_220403.csv')

This is where we should start working from ! 

In [3]:
#now for each of the 12 loan files, we need to map from Zip Codes into FIPs and then take the counts and the 
#sums 

#I used the mapping from Q1 2020, ask Mo and Jess if they're ok with this! 
zip_to_fips_map = pd.read_excel('/Users/zachmariani/ECMA/ECMA_Final_Project/Data/PPP_X/ZIP_COUNTY_032020.xlsx', converters={ "ZIP" : str , "COUNTY" : str })

#this is how I decided to handle conflict between the zip being split between many counties! 
zip_to_fips_map.set_index("ZIP", inplace=True)
zip_to_fips_map.sort_values('TOT_RATIO', inplace=True)
zip_to_fips_map.reset_index(inplace=True)
zip_to_fips_map.drop_duplicates(subset = 'ZIP', keep = 'last', inplace=True)
zip_to_fips_map.set_index('ZIP', inplace=True)

# print(zip_to_fips_map.head())

zip_to_fips_map = zip_to_fips_map['COUNTY'].to_dict()

zip_to_fips_map;

In [47]:
#Jess and Mo, these are the things to choose from 

df.columns

Index(['Unnamed: 0', 'LoanNumber', 'DateApproved', 'SBAOfficeCode',
       'ProcessingMethod', 'BorrowerName', 'BorrowerAddress', 'BorrowerCity',
       'BorrowerState', 'BorrowerZip', 'LoanStatusDate', 'LoanStatus', 'Term',
       'SBAGuarantyPercentage', 'InitialApprovalAmount',
       'CurrentApprovalAmount', 'UndisbursedAmount', 'FranchiseName',
       'ServicingLenderLocationID', 'ServicingLenderName',
       'ServicingLenderAddress', 'ServicingLenderCity', 'ServicingLenderState',
       'ServicingLenderZip', 'RuralUrbanIndicator', 'HubzoneIndicator',
       'LMIIndicator', 'BusinessAgeDescription', 'ProjectCity',
       'ProjectCountyName', 'ProjectState', 'ProjectZip', 'CD', 'JobsReported',
       'NAICSCode', 'Race', 'Ethnicity', 'UTILITIES_PROCEED',
       'PAYROLL_PROCEED', 'MORTGAGE_INTEREST_PROCEED', 'RENT_PROCEED',
       'REFINANCE_EIDL_PROCEED', 'HEALTH_CARE_PROCEED',
       'DEBT_INTEREST_PROCEED', 'BusinessType', 'OriginatingLenderLocationID',
       'OriginatingLender

In [4]:
time_0 = time.time()

dfs_sum = []
dfs_count = []

for n in range(0, 13):
    
    raw_path = '/Users/zachmariani/ECMA/ECMA_Final_Project/Data/PPP_X/ppps/ppp_{}.csv'.format(n)
    df = pd.read_csv(raw_path)
    df = df[df['ProcessingMethod'] == 'PPP']
    cols_of_interest = ['LoanNumber', 'DateApproved', 'BorrowerZip', 'InitialApprovalAmount', 'NAICSCode',\
                        'Race', 'Ethnicity', 'Gender', 'Veteran']
    df = df[cols_of_interest]
    
    df['BorrowerFIP'] = df['BorrowerZip'].str[0:5].astype(str).map(zip_to_fips_map)

    df['DateApproved'] = df['DateApproved'].str[0:2] + '_' + df['DateApproved'].str[-4:]
    
    df = df[['InitialApprovalAmount', 'BorrowerFIP', 'DateApproved']].groupby(['BorrowerFIP', 'DateApproved'])
    
    dfs_sum.append(df.sum())
    dfs_count.append(df.count())

    print('Finished ', n, ' of ', 12, ' in time ', round(time.time() - time_0, 2), ' seconds ')
    time_0 = time.time() 

Finished  0  of  12  in time  11.31  seconds 
Finished  1  of  12  in time  10.14  seconds 
Finished  2  of  12  in time  9.21  seconds 
Finished  3  of  12  in time  9.92  seconds 
Finished  4  of  12  in time  9.03  seconds 
Finished  5  of  12  in time  9.57  seconds 
Finished  6  of  12  in time  9.35  seconds 
Finished  7  of  12  in time  8.83  seconds 
Finished  8  of  12  in time  9.55  seconds 
Finished  9  of  12  in time  9.39  seconds 
Finished  10  of  12  in time  9.14  seconds 
Finished  11  of  12  in time  9.42  seconds 
Finished  12  of  12  in time  9.0  seconds 


In [5]:
grand_sum = reduce(lambda a, b: a.add(b, fill_value=0), dfs_sum)
grand_count = reduce(lambda a, b: a.add(b, fill_value=0), dfs_count)
grand_average = grand_sum/grand_count
grand_average

InitialApprovalAmount
BorrowerFIP DateApproved                       
01001       01_2021                18903.928821
            02_2021                14032.486875
            03_2021                13973.488738
            04_2020                39986.576570
            04_2021                16361.718826
...                                         ...
78030       02_2021               360761.000000
            04_2020               427167.486364
            05_2020               241250.750000
            06_2020               211641.666667
            07_2020               300148.500000

[12909 rows x 1 columns]

In [109]:
#this looks like it was in the right ballpark... 
grand_average.mean()

InitialApprovalAmount    436133.247666
dtype: float64

In [8]:
grand_count

InitialApprovalAmount
BorrowerFIP DateApproved                       
01001       01_2021                       229.0
            02_2021                       384.0
            03_2021                      1236.0
            04_2020                      4259.0
            04_2021                      2113.0
...                                         ...
78030       02_2021                         1.0
            04_2020                        66.0
            05_2020                        16.0
            06_2020                         3.0
            07_2020                         2.0

[12909 rows x 1 columns]

In [16]:
grand_sum.rename(columns={'InitialApprovalAmount':'sum_ppp'}, inplace=True)
grand_count.rename(columns= {'InitialApprovalAmount':'count_ppp'}, inplace=True)
grand_average.rename(columns= {'InitialApprovalAmount':'average_ppp'}, inplace=True)

In [24]:
full_X_df = pd.concat([grand_sum, grand_count, grand_average], axis = 1).reset_index().rename(columns={'BorrowerFIP':'FIPS', 'DateApproved':'date'})

In [25]:
full_X_df

,FIPS,date,sum_ppp,count_ppp,average_ppp
0,01001,01_2021,4.329000e+06,229.0,18903.928821
1,01001,02_2021,5.388475e+06,384.0,14032.486875
2,01001,03_2021,1.727123e+07,1236.0,13973.488738
3,01001,04_2020,1.703028e+08,4259.0,39986.576570
4,01001,04_2021,3.457231e+07,2113.0,16361.718826
...,...,...,...,...,...
12904,78030,02_2021,3.607610e+05,1.0,360761.000000
12905,78030,04_2020,2.819305e+07,66.0,427167.486364
12906,78030,05_2020,3.860012e+06,16.0,241250.750000
12907,78030,06_2020,6.349250e+05,3.0,211641.666667


In [26]:
full_X_df.to_csv('/Users/zachmariani/ECMA/ECMA_Final_Project/Data/PPP_X/clean_X.csv')

In [ ]:
#Some extra exploring below!

In [80]:
df[['InitialApprovalAmount', 'BorrowerFIP', 'DateApproved']].groupby(['BorrowerFIP', 'DateApproved'])

In [75]:
df.groupby(['BorrowerFIP', 'DateApproved']).count()

LoanNumber  BorrowerZip  InitialApprovalAmount  \
BorrowerFIP DateApproved                                                   
01001       01_2021                1            1                      1   
            04_2020               47           47                     47   
            04_2021                1            1                      1   
            06_2020                2            2                      2   
            07_2020                1            1                      1   
...                              ...          ...                    ...   
78030       02_2021                1            1                      1   
            04_2020               66           66                     66   
            05_2020               16           16                     16   
            06_2020                3            3                      3   
            07_2020                2            2                      2   

                          NAICSCode  Race  Ethnicity  Gender  Veteran  
BorrowerFIP DateApproved                                               
01001       01_2021               1     1          1       1        1  
            04_2020              47    47         47      47       47  
            04_2021               1     1          1       1        1  
            06_2020               2     2          2       2        2  
            07_2020               1     1          1       1        1  
...                             ...   ...        ...     ...      ...  
78030       02_2021               1     1          1       1        1  
            04_2020              66    66         66      66       66  
            05_2020              15    16         16      16       16  
            06_2020               3     3          3       3        3  
            07_2020               2     2          2       2        2  

[11531 rows x 8 columns]

In [68]:
df.head()

,LoanNumber,DateApproved,BorrowerZip,InitialApprovalAmount,NAICSCode,Race,Ethnicity,Gender,Veteran
0,9547507704,05_2020,29150-9662,769358.78,325510.0,Unanswered,Unknown/NotStated,Unanswered,Unanswered
1,9777677704,05_2020,29420-9000,736927.79,561730.0,White,Unknown/NotStated,Male Owned,Non-Veteran
2,5791407702,05_2020,98112-2922,691355.00,NaN,Unanswered,Unknown/NotStated,Unanswered,Unanswered
3,6223567700,05_2020,92407-1740,499871.00,236115.0,American Indian or Alaska Native,Not Hispanic or Latino,Male Owned,Non-Veteran
4,9662437702,05_2020,NaN,367437.00,484210.0,Unanswered,Unknown/NotStated,Unanswered,Unanswered
